In [3]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm_notebook
import warnings
warnings.filterwarnings("ignore")
import random
import math
import pickle
from datetime import datetime, timedelta
from sklearn.tree import DecisionTreeRegressor
# from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier

# from skopt import gp_minimize
# from skopt.space import Integer, Real, Categorical
# from skopt.utils import use_named_args

In [19]:
# close = pd.read_parquet('rus_close_201120.parquet')#.iloc[:,:10]
# volume = pd.read_parquet('rus_volume_201120.parquet')#.iloc[:,:10]

market = pd.read_csv("market.csv", index=False)

TypeError: read_csv() got an unexpected keyword argument 'index'

In [17]:
market

,date,vix,spy,spyv,rate10y,rate3m,dxy,oil
0,02.01.2002,22.71,115.53,18652200,5.160,1.736,115.790,21.01
1,03.01.2002,21.34,116.84,15743100,5.111,1.716,116.110,20.37
2,04.01.2002,20.45,117.62,20140700,5.128,1.715,116.330,21.62
3,07.01.2002,21.94,116.79,13106500,5.049,1.685,116.330,21.48
4,08.01.2002,21.83,116.52,12687200,5.080,1.675,116.830,21.25
...,...,...,...,...,...,...,...,...
4808,08.02.2021,21.24,390.51,38365185,1.171,0.030,90.934,57.97
4809,09.02.2021,21.63,390.25,35551059,1.159,0.030,90.439,58.36
4810,10.02.2021,21.99,390.08,59154370,1.123,0.041,90.371,58.68
4811,11.02.2021,21.25,390.71,42913288,1.166,0.038,90.417,58.24


In [23]:
close.shape

(4632, 14106)

In [20]:
market = market.dropna()

In [22]:
def year_start(date):
    """
    Makes the model aware of current date
    """
    if date.year % 4 == 0:
        base = 366
    else:
        base = 365
        
    ystart = pd.to_datetime(str(date.year) + '-01-01')
    return (date - ystart).days / base


def get_tick_df(x, split=pd.to_datetime('2017-01-01')):
    """
    Combines information for a ticker from 'close' and 'volume' dataframes into one,
    Calculates 'valtr' column - 'value traded',
    Splits data into master 'train' and 'test' parts
    
    x - column index for the respective column date
    split - date to split into 'test' and 'train' parts
    """
    tick_df = close.iloc[:,x:x+2]
    tick = tick_df.columns[1]
    tick_df['volume'] = volume[tick]
    tick_df.columns = ['date', tick, 'volume']
    tick_df = tick_df.set_index('date').dropna()
    tick_df['valtr'] = tick_df[tick] * tick_df['volume'] / 1000000
    train = tick_df.loc[:split]
    test = tick_df.copy()
    
    return train, test

market['dt'] = market.index
market['dt'] = market['dt'].apply(year_start)
market['spyval'] = market['spy'] * market['spyv'] / 1000000

AttributeError: 'int' object has no attribute 'year'

In [5]:
x = 0

train_df = {}
test_df = {}
split = pd.to_datetime('2017-01-01')

for _ in tqdm_notebook(range(int(close.shape[1]/2))):
    
    tick = close.iloc[:,x+1].name
    if close[tick].dropna().shape[0] == 0:
        # some are empty, these are ignored
        x+=2
        continue
    else:
        train_df[tick], test_df[tick] = get_tick_df(x, split=split)
        x+=2

train_market = market.loc[:split]
test_market = market.copy()

train_market.index[-1], test_market.index[0]

  0%|          | 0/7053 [00:00<?, ?it/s]

(Timestamp('2016-12-30 00:00:00'), Timestamp('2002-01-02 00:00:00'))

In [12]:
def check_cv_error(cv_periods, verbose=True):
    """
    Checks for look-ahead biases and other errors in cv periods,
    reports data if required so or if silent - raises exceptions
    """
    sub_start = cv_periods['sub_start']
    sub_end = cv_periods['sub_end']
    val_start = cv_periods['val_start']
    val_end = cv_periods['val_end']
#     test_start = cv_periods['test_start']
#     test_end = cv_periods['test_end']
    
    if verbose:
        if sub_start > sub_end:
            print("Training sample dates error")

        if sub_end > val_start:
            print("Validation starts before training ends!")

        if val_start > val_end:
            print("Validation sample dates error")

#         if val_end > test_start:
#             print("Testing starts before validation ends!")

#         if test_start > test_end:
#             print("Testing sample dates error")

        print("Training starts at", sub_start, "Training sample size is", (sub_end - sub_start).days, 
              "\nValidation starts at", val_start, "Validation sample size is", (val_end - val_start).days,
#              "\nTesting starts at", test_start, "Testing sample size is", (test_end - test_start).days,
              "\nNumber of days purged between training and validation is", (val_start - sub_end).days)

#              "\nNumber of days purged between validation and testing is", (test_start - val_end).days
    
    else:
        if sub_start > sub_end:
            raise Exception("Training sample dates error")

        if sub_end > val_start:
            raise Exception("Validation starts before training ends!")

        if val_start > val_end:
            raise Exception("Validation sample dates error")

#         if val_end > test_start:
#             raise Exception("Testing starts before validation ends!")

#         if test_start > test_end:
#             raise Exception("Testing sample dates error")
        
    pass

def shift_cv(cv_periods, shift=6, drag=False):
    """
    Shifts cross-validation periods by a set number of months
    """ 
    
    new_periods = {}
    for k, v in cv_periods.items():
        if (k == 'sub_start') and (drag == False):
            new_periods[k] = v
        else:
            new_periods[k] = (v + np.timedelta64(shift, 'M')).round('d')
    
    return new_periods

In [13]:
def zero_to_one(series, log=True):
    """
    Scales data from 0 to 1 while not looking ahead because of cummin/cummax methods.
    If log=True, first takes np.log of the series
    """
    if log:
        return (np.log(series) - np.log(series).cummin()) / (np.log(series).cummax() - np.log(series).cummin() )
    else:
        return (series - series.cummin()) / (series.cummax() - series.cummin() )
    
def prod_return(valdf, compress=0.5, sharpe=True, com=0.002, penalize=0.1):
    
    if 'predy' not in valdf.columns:
        print("No 'predy' column found!")
        return 0
    elif 'fwd' not in valdf.columns:
        print("No 'fwd' column found!")
        return 0
    elif (compress != None) and ( compress <= 0):
        print("'compress' input must be >0!")
        return 0
    
    else:
        if compress != None:
            valdf['fwd'] = np.where( valdf['fwd'] > 0.5, 0.5, valdf['fwd'] )
        
        valdf['pred_return'] = 1 + valdf['predy'] * (valdf['fwd'] - com)
        
        if valdf.predy.sum() >= 1:
            prod_return = valdf['pred_return'].prod()**(1.0/len(valdf[valdf.predy == 1])) - 1
        else:
            return 0
        
        if penalize:
            if 'testy' not in valdf.columns:
                print("No 'testy' column found!")
                return 0
            
            ratio = valdf['predy'].mean()/valdf['testy'].mean()
            if ratio < penalize:
                prod_return = prod_return * ratio / penalize
        
        if sharpe:
            
            return (prod_return / valdf[valdf.predy == 1]['fwd'].std())
            
        else:
            return prod_return

def preproc_mkt(mkt_df, price_lags=[1,5,20,250], vol_lags=[2,10]):
    
    """
    Prepares data for market dataframe. 
    Creates delta features for every price_lags unit for spy price, 10year rate, 3month rate and oil price.
    Creates rolling average features for 1 and every vol_lags unit for vix, dollar index and spy volume. 
    
    returns clean market dataframe and columns of the dataframe
    Attention! returing dataframe contains a non-feature column 'spyv', used for further relative feature creation 
    """
    
    for lag in price_lags:
        ret_name = 'spy' + str(lag)
        mkt_df[ret_name] = mkt_df['spy']/mkt_df['spy'].shift(lag)-1
        mkt_df[ret_name] = zero_to_one(mkt_df[ret_name], log=False)

        ret_name = '10y' + str(lag)
        mkt_df[ret_name] = mkt_df['rate10y']/mkt_df['rate10y'].shift(lag)-1
        mkt_df[ret_name] = zero_to_one(mkt_df[ret_name], log=False)

        ret_name = '3m' + str(lag)
        mkt_df[ret_name] = mkt_df['rate3m']/mkt_df['rate3m'].shift(lag)-1
        mkt_df[ret_name] = zero_to_one(mkt_df[ret_name], log=False)

        ret_name = 'oil' + str(lag)
        mkt_df[ret_name] = mkt_df['oil']/mkt_df['oil'].shift(lag)-1
        mkt_df[ret_name] = zero_to_one(mkt_df[ret_name], log=False)


    mkt_df['spyvol'] = zero_to_one(mkt_df['spyv'], log=True)
    
    mkt_df['dxy'] = zero_to_one(mkt_df['dxy'], log=False)
    
    mkt_df['s_vix'] = zero_to_one(mkt_df['vix'], log=True)

    for lag in vol_lags:
        mkt_df['spyvol' + str(lag)] = mkt_df['spyvol'].rolling(lag).mean()
        mkt_df['dxy' + str(lag)] = mkt_df['dxy'].rolling(lag).mean()
        mkt_df['s_vix' + str(lag)] = mkt_df['s_vix'].rolling(lag).mean()

    junk_cols = ['vix', 'spy', 'spyv', 'rate10y', 'rate3m', 'oil']

    mkt_df = mkt_df.drop(columns=junk_cols)
#    print(mkt_df.shape)
    return mkt_df, mkt_df.columns

def preproc_tick(tick, fulldf, market_df, market_cols, fwd=5, 
                 cut_first=150, price_lags=[1,2,5,10,20,50,100,250], vol_lags=[2,5,10,20]):
    """
    Prepares data for a ticker dataframe. 
    Creates delta features for every price_lags unit for close price.
    Creates rolling average features for 1 and every vol_lags unit for volatility and value traded. 
    Cuts first 150 days, because they are too noisy usually due to cummax/cummin methods
    
    Parameters:
        tick - ticker used
        fulldf - dictionary with all tickers that is limited by cv_periods
        market_df - output of preproc_mkt function
        market_cols - output of preproc_mkt function
        fwd - forecasting period in days
    
    returns clean ticker dataframe 
    """
    
    idf = fulldf[tick]
    idf['fwd'] = idf[tick].shift(-fwd) / idf[tick] - 1
    idf['ycol'] = np.where( idf['fwd'] >= 0.02, 1, 0 )
    
    idf[market_cols] = market_df
    
    for lag in price_lags:
        ret_name = 'ret' + str(lag)
        idf[ret_name] = idf[tick]/idf[tick].shift(lag)-1
        idf[ret_name] = zero_to_one(idf[ret_name], log=False)

    idf['relvol'] = np.log(idf['valtr']) / np.log(idf['spyval'])
#    idf['relvol'] = zero_to_one(idf['relvol'], log=False)
    idf['vol'] = zero_to_one(idf['volume'], log=True)
    idf['val'] = zero_to_one(idf['valtr'], log=True)

    for lag in vol_lags:
        idf['vol' + str(lag)] = idf['vol'].rolling(lag).mean()
        idf['val' + str(lag)] = idf['val'].rolling(lag).mean()
        idf['rel_val' + str(lag)] = idf['relvol'].rolling(lag).mean()
    
    idf['tick'] = tick
    
    junk_cols = [tick, 'volume', 'valtr', 'spyval']
    
#     print(tick, idf.drop(columns=junk_cols).iloc[cut_first:].dropna().shape, 
#           idf.drop(columns=junk_cols).iloc[cut_first:].isna().sum(), 
#           '\n', idf.drop(columns=junk_cols).iloc[cut_first:], '\n\n\n\n')

#    print(idf.drop(columns=junk_cols).iloc[cut_first:].dropna().shape)
    return idf.drop(columns=junk_cols).iloc[cut_first:].dropna()
    

In [14]:
def prepare_tvt(train_df, train_market, cv_periods, settings=None, state_size=False):
    """
    creates train, validation and test subsamples for the given cv_periods
    Parameters:
        train_df - full training dictionary of all tickers
        train_df - full market training dictionary
        cv_periods - current limits on dates
    """
    sub_df = {}
    val_df = {}
#     tst_df = {}

    for k, v in train_df.items():

        sub_df[k] = v.loc[cv_periods['sub_start']:cv_periods['sub_end']]
        val_df[k] = v.loc[cv_periods['sub_start']:cv_periods['val_end']]
#         tst_df[k] = v.loc[cv_periods['sub_start']:cv_periods['test_end']]

    market_sub = train_market.loc[cv_periods['sub_start']:cv_periods['sub_end']].copy()
    market_val = train_market.loc[cv_periods['sub_start']:cv_periods['val_end']].copy()
#     market_tst = train_market.loc[cv_periods['sub_start']:cv_periods['test_end']].copy()
    
    if state_size:
        print(market_sub.shape, market_sub.isna().sum())
    
    if settings == None:
        market_sub, _ = preproc_mkt(market_sub)
        market_val, market_cols = preproc_mkt(market_val)
#         market_tst, market_cols = preproc_mkt(market_tst)
    else:
        market_sub, _ = preproc_mkt(market_sub, **settings['preproc_mkt'])
        market_val, market_cols = preproc_mkt(market_val, **settings['preproc_mkt'])
#         market_tst, market_cols = preproc_mkt(market_tst, **settings['preproc_mkt'])        

    
    train = {}
    val = {}
#     tst = {}
    
    if settings == None:
        for tick in tqdm_notebook(sub_df.copy().keys()):
            train[tick] = preproc_tick(tick, sub_df, market_sub, market_cols)
            val[tick] = preproc_tick(tick, val_df, market_val, market_cols).loc[cv_periods['val_start']:]
#             tst[tick] = preproc_tick(tick, tst_df, market_tst, market_cols).loc[cv_periods['test_start']:]
    else:
        for tick in sub_df.copy().keys():
            train[tick] = preproc_tick(tick, sub_df, market_sub, market_cols, **settings['preproc_tick'])
            val[tick] = preproc_tick(tick, val_df, market_val, market_cols, \
                                     **settings['preproc_tick']).loc[cv_periods['val_start']:]
#             tst[tick] = preproc_tick(tick, tst_df, market_tst, market_cols, \
#                                      **settings['preproc_tick']).loc[cv_periods['test_start']:]        
    
    train = pd.concat( train.values() ).sort_index()
    train['tick date'] = train['tick'] + ':' + train.index.astype(str)
    train = train.set_index('tick date').drop(columns=['tick'])
    
    val = pd.concat( val.values() ).sort_index()
    val['tick date'] = val['tick'] + ':' + val.index.astype(str)
    val = val.set_index('tick date').drop(columns=['tick'])
    
#     tst = pd.concat( tst.values() ).sort_index()
#     tst['tick date'] = tst['tick'] + ':' + tst.index.astype(str)
#     tst = tst.set_index('tick date').drop(columns=['tick'])
    
    if state_size:
        print(train.shape)
    
    return train, val #, tst

In [15]:
def cv_step(train_df, train_market, cv_periods, settings=None):
    """
    Calculates validation score, provides with testing dataframe and current model
    for current cross-validation step.
    Parameters:
        train_df - full training dictionary of all tickers
        train_df - full market training dictionary
        cv_periods - current limits on dates
        settings - settings to pass on    
    """
    
    if settings == None:
        genmodel = LGBMClassifier
        check_cv_error(cv_periods, verbose=False)
        model = genmodel(random_state=1, n_jobs=-1)
        train, val = prepare_tvt(train_df, train_market, cv_periods)
    else:
        genmodel = settings['genmodel']
        check_cv_error(cv_periods, verbose=settings['verbose'])
        train, val = prepare_tvt(train_df, train_market, cv_periods, settings)    
        
        model = genmodel(random_state=1, n_jobs=-1, **settings['model_settings'])
        
    
    
    train_X = train.drop(columns = ['ycol', 'fwd'])
    train_y = train['ycol']
    val_X = val.drop(columns = ['ycol', 'fwd'])
    val_y = val['ycol']
    
    try:
        model.fit(train_X, train_y)
    except:
        train, val = prepare_tvt(train_df, train_market, cv_periods, settings, state_size=True)
        print(train_X.shape, train.shape)
        print(settings['model_settings'])
        print(cv_periods)
        
        raise Exception('Error')
    
    if settings == None:
        val_X['proby'] = model.predict_proba(val_X)[:,1]
        val_X['predy'] = np.where(val_X['proby'] > 0.5, 1, 0)
    else:
        val_X['proby'] = model.predict_proba(val_X)[:,1]
        val_X['predy'] = np.where(val_X['proby'] > settings['threshold'], 1, 0)
   
    val_X['fwd'] = val['fwd']
        
    val_X['testy'] = val_y
    #val_score = val_X[['predy', 'testy']].corr().iloc[0][1]
    
    return val_X[['testy','predy','proby','fwd']], model

In [10]:
cv_periods = {
        'sub_start': pd.to_datetime('2002-07-01'),
        'sub_end': pd.to_datetime('2005-07-01'),
        'val_start': pd.to_datetime('2005-08-01'),
        'val_end': pd.to_datetime('2006-01-01'),
#         'test_start': pd.to_datetime('2006-02-01'),
#         'test_end':pd.to_datetime('2006-07-01')
    }

val, model = cv_step(train_df, train_market, cv_periods)

  0%|          | 0/6949 [00:00<?, ?it/s]

In [11]:


search_space = [
    
    Categorical(['gbdt', 'dart', 'goss', 'rf'], name='boosting_type'),
    Integer(2, 4096, name='num_leaves'),
    Real(0.0001, 0.3, prior='log-uniform', name='learning_rate'),
    Integer(2, 1000, name='n_estimators'),
    Real(0.01, 0.999, name='subsample'),
    Real(0.0001, 0.3, prior='log-uniform', name='min_child_weight'),
    Integer(5, 1000, name='min_child_samples'),
    Real(0.2, 0.999, prior='uniform', name='feature_fraction'),
    Integer(2, 15, name='fwd'),
    Real(0.5, 0.9, prior='uniform', name='threshold'),
    Categorical([None, 'balanced'], name='class_weight'),
    Categorical([True, False], name='drag'),
]

# search_space = [
#     Integer(1,5, name='asd'),
#     Categorical(['a','b'], name='letter')
# ]

In [12]:
@use_named_args(search_space)
def cv_round(**opt_sets):
    
    val_scores = []
    models = []

    cv_periods = {
        'sub_start': pd.to_datetime('2002-07-01'),
        'sub_end': pd.to_datetime('2005-07-01'),
        'val_start': pd.to_datetime('2005-08-01'),
        'val_end': pd.to_datetime('2006-01-01'),
#         'test_start': pd.to_datetime('2006-02-01'),
#         'test_end':pd.to_datetime('2006-07-01')
    }
    
    settings = {
        'genmodel': LGBMClassifier,
        'verbose': False,
#         'force_cw': False, 
        'model_settings': {
            'boosting_type': 'gbdt',
            'num_leaves': 31,
            'max_depth': -1,
            'learning_rate': 0.1,
            'n_estimators': 100,
            'subsample_for_bin': 200000,
            'objective': None,
            'class_weight': None, #'balanced'
            'min_split_gain': 0.0,
            'min_child_weight': 0.001,
            'min_child_samples': 20,
            'subsample': 1.0,
            'subsample_freq': 1,
            'colsample_bytree': 1.0,
            'reg_alpha': 0.0,
            'reg_lambda': 0.0,
            'importance_type': 'split'
        },
        'preproc_mkt': {
            'price_lags': [1,5,20,250], 
            'vol_lags': [2,10]
        },
        'preproc_tick': {
            'fwd': 10,
            'cut_first': 150, 
            'price_lags': [1,2,5,10,20,50,100,250], 
            'vol_lags': [2,5,10,20]
        }, 
        'threshold': 0.5,
        'drag': False
    }
    
    if opt_sets != None:
        
        if ('boosting_type' in opt_sets.keys()) and (opt_sets['boosting_type'] == 'goss') and \
            ('subsample' in opt_sets.keys()):
            del opt_sets['subsample']
            
        
        for k, v in opt_sets.items():
            
            if k in settings.keys():
                settings[k] = v
            elif k in settings['preproc_tick'].keys():
                settings['preproc_tick'][k] = v
            else:
                settings['model_settings'][k] = v

    
    while cv_periods['val_end'] < split:
        try:
            valdf, _, = cv_step(train_df, train_market, cv_periods, settings)
            val_scores.append(valdf)
        except:
            print(opt_sets)
            raise Exception("Error")
        cv_periods = shift_cv(cv_periods, drag=settings['drag'])
    
    score = -prod_return(pd.concat(val_scores), sharpe=False)/settings['preproc_tick']['fwd']
    #print(score)
    
    myfile = open('working.txt', 'w')
    myfile.write(str(-score) + '\n' + str(settings) + '\n\n' + datetime.now().strftime("%H:%M:%S"))
    myfile.close()
    
    if np.isnan(score):
        return 0
    else:
        return score 

In [15]:
result = gp_minimize(cv_round, search_space, random_state=1, n_calls=10, verbose=True)

Iteration No: 1 started. Evaluating function at random point.
[LightGBM] [Warning] feature_fraction is set=0.7351270834059807, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7351270834059807
[LightGBM] [Warning] feature_fraction is set=0.7351270834059807, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7351270834059807
{'boosting_type': 'rf', 'num_leaves': 3820, 'learning_rate': 0.0002789357895948863, 'n_estimators': 999, 'subsample': 0.24349199820550438, 'min_child_weight': 0.0023930913450846, 'min_child_samples': 391, 'feature_fraction': 0.7351270834059807, 'fwd': 14, 'threshold': 0.8385243666744069, 'class_weight': None, 'drag': True}


Exception: Error

In [ ]:
result.x


In [ ]:
search_space = [
    
    Categorical(['gbdt', 'dart', 'goss', 'rf'], name='boosting_type'),
    Integer(2, 4096, name='num_leaves'),
    Real(0.0001, 0.3, prior='log-uniform', name='learning_rate'),
    Integer(2, 1000, name='n_estimators'),
    Real(0.01, 0.999, name='subsample'),
    Real(0.0001, 0.3, prior='log-uniform', name='min_child_weight'),
    Integer(5, 1000, name='min_child_samples'),
    Real(0.2, 0.999, prior='uniform', name='feature_fraction'),
    Integer(2, 15, name='fwd'),
    Real(0.5, 0.85, prior='uniform', name='threshold'),
    Categorical([None, 'balanced'], name='class_weight'),
    Categorical([True, False], name='drag'),
]

opt_sets = {}
x = 0

for opt in result.x:
    opt_sets[search_space[x].name] = opt
    x+=1
    

In [ ]:
cv_periods = {
    'sub_start': pd.to_datetime('2013-12-01'),
    'sub_end': pd.to_datetime('2016-12-01'),
    'val_start': pd.to_datetime('2017-01-01'),
    'val_end': pd.to_datetime('2017-04-01'),
#         'test_start': pd.to_datetime('2006-02-01'),
#         'test_end':pd.to_datetime('2006-07-01')
}

settings = {
    'genmodel': LGBMClassifier,
    'verbose': True,
#         'force_cw': False, 
    'model_settings': {
        'boosting_type': 'gbdt',
        'num_leaves': 31,
        'max_depth': -1,
        'learning_rate': 0.1,
        'n_estimators': 100,
        'subsample_for_bin': 200000,
        'objective': None,
        'class_weight': None, #'balanced'
        'min_split_gain': 0.0,
        'min_child_weight': 0.001,
        'min_child_samples': 20,
        'subsample': 1.0,
        'subsample_freq': 1,
        'colsample_bytree': 1.0,
        'reg_alpha': 0.0,
        'reg_lambda': 0.0,
        'importance_type': 'split'
    },
    'preproc_mkt': {
        'price_lags': [1,5,20,250], 
        'vol_lags': [2,10]
    },
    'preproc_tick': {
        'fwd': 10,
        'cut_first': 150, 
        'price_lags': [1,2,5,10,20,50,100,250], 
        'vol_lags': [2,5,10,20]
    }, 
    'threshold': 0.5,
    'drag': False
}

if ('boosting_type' in opt_sets.keys()) and (opt_sets['boosting_type'] == 'goss') and \
    ('subsample' in opt_sets.keys()):
    del opt_sets['subsample']

for k, v in opt_sets.items():
            
    if k in settings.keys():
        settings[k] = v
    elif k in settings['preproc_tick'].keys():
        settings['preproc_tick'][k] = v
    else:
        settings['model_settings'][k] = v

val_dfs = []

while cv_periods['val_end'] < test_df['aa'].index[-1]:
    
    cv_periods = shift_cv(cv_periods, shift=3, drag=True)
    #print(cv_periods['val_end'])
    val, _ = cv_step(test_df, test_market, cv_periods, settings=settings)
    val_dfs.append(val)

#prod_return(val, penalize=False, sharpe=False, compress=None)

In [ ]:
prod_return(pd.concat(val_dfs), penalize=False, sharpe=False, compress=None, com=0.002)

In [ ]:
df = pd.concat(val_dfs)
print(df[df.predy == 1].shape)
df[df.predy == 1]['fwd'].hist(bins=40, figsize=[15,10])